In [1]:
import numpy as np
from utils.mdrp import mdrp_prob
import cyipopt

In [2]:
num_i = 5
num_j = 3

prob_setup = {}

prob_setup['s_time']   = np.random.rand(num_i, num_j)*5/60
prob_setup['t_time']   = np.random.rand(num_i, num_j)*30/60
prob_setup['cost']     = np.random.rand(num_i, num_j)*20/60
prob_setup['beta']     = 0.1 + np.random.rand(num_i, num_j)*0.4
prob_setup['N']        = np.ones(num_j)*10
prob_setup['k']        = np.ones(num_j)*10/60
prob_setup['mu']       = np.array([6,3,2])
prob_setup['max_rho']  = 0.9
prob_setup['max_cost'] = 10000

for param in prob_setup.keys():
    print('Parameter: %s \nValue:\n'%param,prob_setup[param])
    print('-----------------------------')

mdrp = mdrp_prob(prob_setup)

Parameter: s_time 
Value:
 [[0.08284401 0.06970119 0.03393289]
 [0.00308309 0.07981029 0.02400714]
 [0.05791667 0.07990427 0.07202951]
 [0.07020401 0.03683558 0.0005772 ]
 [0.07890801 0.05750366 0.07934389]]
-----------------------------
Parameter: t_time 
Value:
 [[0.01896479 0.00268778 0.08627038]
 [0.01677025 0.07527797 0.38522996]
 [0.42468477 0.40707207 0.11422641]
 [0.07694539 0.14461213 0.13751309]
 [0.1396678  0.44163688 0.24154447]]
-----------------------------
Parameter: cost 
Value:
 [[0.05453874 0.10865615 0.27904188]
 [0.01406719 0.28603461 0.06085958]
 [0.16046562 0.10614809 0.3186182 ]
 [0.28494555 0.10565452 0.01459762]
 [0.04749432 0.05306493 0.0498692 ]]
-----------------------------
Parameter: beta 
Value:
 [[0.38991064 0.32448775 0.19044263]
 [0.12071209 0.29882491 0.38030028]
 [0.31191651 0.19539372 0.44708033]
 [0.27240894 0.1080664  0.13656497]
 [0.22527017 0.20317609 0.33259016]]
-----------------------------
Parameter: N 
Value:
 [10. 10. 10.]
----------------

In [3]:
x = mdrp.guess_init()

obj = mdrp.objective(x)
print(obj)

grad = mdrp.gradient(x)
print(grad)

cons = mdrp.constraints(x)
print(cons)

jac = mdrp.jacobian(x)
print(jac)

hess = mdrp.hessian(x,lagrange=None,obj_factor=1)
print(hess)



0.2840910607981419
[0.02710521 0.02222019 0.03513788 0.0189317  0.03925042 0.08858763
 0.10453193 0.10848471 0.0431889  0.03828814 0.05206282 0.04127214
 0.05385497 0.11063353 0.07164593]
[0.64801873 0.02777778 0.05555556 0.08333333 1.         1.
 1.         1.         1.        ]
[[0.05453874 0.10865615 0.27904188 0.01406719 0.28603461 0.06085958
  0.16046562 0.10614809 0.3186182  0.28494555 0.10565452 0.01459762
  0.04749432 0.05306493 0.0498692 ]
 [0.01666667 0.         0.         0.01666667 0.         0.
  0.01666667 0.         0.         0.01666667 0.         0.
  0.01666667 0.         0.        ]
 [0.         0.03333333 0.         0.         0.03333333 0.
  0.         0.03333333 0.         0.         0.03333333 0.
  0.         0.03333333 0.        ]
 [0.         0.         0.05       0.         0.         0.05
  0.         0.         0.05       0.         0.         0.05
  0.         0.         0.05      ]
 [1.         1.         1.         0.         0.         0.
  0.         0

In [4]:
lb, ub, cl, cu = mdrp.get_bounds()
x0 = mdrp.guess_init()

nlp = cyipopt.Problem(
    n=len(x0),
    m=len(cl),
    problem_obj=mdrp,
    lb=lb,
    ub=ub,
    cl=cl,
    cu=cu
    )

# nlp.addOption('derivative_test', 'second-order')
nlp.add_option('mu_strategy', 'adaptive')
nlp.add_option('tol', 1e-7)

#
# Scale the problem (Just for demonstration purposes)
#
# nlp.set_problem_scaling(
#     obj_scaling=1,
#     x_scaling=[1, 1, 1, 1]
#     )
# nlp.add_option('nlp_scaling_method', 'user-scaling')

#
# Solve the problem
#
x, info = nlp.solve(x0)

print("Solution of the primal variables: x=%s\n" % repr(x))

print("Solution of the dual variables: lambda=%s\n" % repr(info['mult_g']))

print("Objective=%s\n" % repr(info['obj_val']))


******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit https://github.com/coin-or/Ipopt
******************************************************************************

This is Ipopt version 3.14.11, running with linear solver MUMPS 5.2.1.

Number of nonzeros in equality constraint Jacobian...:       75
Number of nonzeros in inequality constraint Jacobian.:       60
Number of nonzeros in Lagrangian Hessian.............:      120

Objective value at iteration #0 is - 0.284091
Total number of variables............................:       15
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       15
                     variables with only upper bounds:        0
Total number of equality constraints..........